In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [3]:
housing = pd.read_csv("/kaggle/input/clean-dataset-house-pricing/clean_data.csv")
df=housing
learning_rate = 1e-3
n_iters = 10000

In [6]:
print(housing)

        price      area  bedrooms  bathrooms   stories  mainroad  guestroom  \
0    13300000  0.396564       0.6   0.333333  0.666667         1          0   
1    12250000  0.502405       0.6   1.000000  1.000000         1          0   
2    12250000  0.571134       0.4   0.333333  0.333333         1          0   
3    12215000  0.402062       0.6   0.333333  0.333333         1          0   
4    11410000  0.396564       0.6   0.000000  0.333333         1          1   
..        ...       ...       ...        ...       ...       ...        ...   
540   1820000  0.092784       0.2   0.000000  0.000000         1          0   
541   1767150  0.051546       0.4   0.000000  0.000000         0          0   
542   1750000  0.135395       0.2   0.000000  0.000000         1          0   
543   1750000  0.086598       0.4   0.000000  0.000000         0          0   
544   1750000  0.151203       0.4   0.000000  0.333333         1          0   

     basement  hotwaterheating  airconditioning   p

In [5]:
target_col = df.columns[0]

# Xác định các cột liên tục (numeric) và phân loại (object)
continuous_cols = df.select_dtypes(include=['float64', 'int64']).columns.drop(target_col, errors='ignore')
categorical_cols = df.select_dtypes(include=['object']).columns

# Chuẩn hóa các cột liên tục
# scaler = StandardScaler()
scaler = MinMaxScaler()
df[continuous_cols] = scaler.fit_transform(df[continuous_cols])

# Label encode các cột phân loại
le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

In [7]:
# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2, random_state=42
)

In [8]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            y_predicted = np.dot(X, self.weights) + self.bias

            # Tính đạo hàm
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Cập nhật trọng số
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

In [42]:
# Huấn luyện mô hình
reg = LinearRegression(learning_rate=learning_rate, n_iters=10000)
reg.fit(X_train, y_train)

# Dự đoán trên dữ liệu test
predictions = reg.predict(X_test)

# Tính các thông số đánh giá
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# Hiển thị các thông số đánh giá
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R-squared: {r2:.2f}')

Mean Squared Error (MSE): 2048385386436.48
Mean Absolute Error (MAE): 1015852.35
R-squared: 0.59


In [50]:
import torch
import torch.nn as nn
import torch.optim as optim

# Chuyển dữ liệu thành tensor
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Xây dựng mô hình ANN
class ANN_Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(ANN_Model, self).__init__()
        self.hidden_layer_1 = nn.Linear(input_size, 64)
        self.hidden_layer_2 = nn.Linear(64,32)
        self.hidden_layer_3 = nn.Linear(32,16)
        self.output_layer = nn.Linear(16, output_size)
        
    def forward(self, x):
        x = torch.relu(self.hidden_layer_1(x))
        x = torch.relu(self.hidden_layer_2(x))
        x = torch.relu(self.hidden_layer_3(x))
        x = self.output_layer(x)
        return x

    def predict(self, x):
        x = torch.tensor(x.values, dtype=torch.float32)
        output = self.forward(x)  
        return output.detach().numpy().tolist()[0]

# Khởi tạo mô hình ANN
input_size = X_train.shape[1]  # Số lượng đặc trưng
output_size = 1  # Vì đây là bài toán hồi quy

ann_model = ANN_Model(input_size, output_size)

# Định nghĩa loss function và optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(ann_model.parameters(), lr=1e-3)

# Huấn luyện mô hình
n_epochs = 10000
for epoch in range(n_epochs):
    ann_model.train()
    
    # Dự đoán và tính loss
    y_pred = ann_model(X_train_tensor)
    loss = criterion(y_pred, y_train_tensor)
    
    # Gradient và cập nhật weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # if (epoch + 1) % 100 == 0:
    #     print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Dự đoán trên dữ liệu test
ann_model.eval()  # Chuyển mô hình sang chế độ đánh giá
with torch.no_grad():
    y_test_pred = ann_model(X_test_tensor)

# Tính các thông số đánh giá
mse = mean_squared_error(y_test_tensor.numpy(), y_test_pred.numpy())
mae = mean_absolute_error(y_test_tensor.numpy(), y_test_pred.numpy())
r2 = r2_score(y_test_tensor.numpy(), y_test_pred.numpy())

# In các thông số đánh giá
print(f'Mean Squared Error: {mse:.2f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'R-squared: {r2:.2f}')

Mean Squared Error: 1759458230272.00
Mean Absolute Error: 970413.06
R-squared: 0.65


In [43]:
class RandomForest:
    def __init__(self, n_estimators=1000, max_depth=None, random_state=42):
        self.model = RandomForestRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=random_state
        )

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        return self.model.predict(X_test)

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        return mse, mae, r2

# Huấn luyện mô hình Random Forest
rf_model = RandomForest(n_estimators=100, max_depth=10)
rf_model.train(X_train, y_train)
mse, mae, r2 = rf_model.evaluate(X_test, y_test)

print(f"Mean Squared Error: {mse:.4f}")
print(f"Mean Absolute Error: {mae:.4f}")
print(f"R-squared: {r2:.4f}")

Mean Squared Error: 1980684038933.8408
Mean Absolute Error: 1026163.5980
R-squared: 0.6081


In [44]:
from xgboost import XGBRegressor

class XGBoostModel:
    def __init__(self, n_estimators=1000, learning_rate=0.001, max_depth=6, random_state=42):
        self.model = XGBRegressor(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            random_state=random_state,
            objective='reg:squarederror'
        )

    def train(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        return self.model.predict(X_test)

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f"Mean Squared Error: {mse:.4f}")
        print(f"Mean Absolute Error: {mae:.4f}")
        print(f"R-squared: {r2:.4f}")
        return mse, mae, r2

# Huấn luyện mô hình XGBoost
xgb_model = XGBoostModel(n_estimators=100, learning_rate=0.1, max_depth=6)
xgb_model.train(X_train, y_train)

# Đánh giá mô hình
mse, mae, r2 = xgb_model.evaluate(X_test, y_test)

Mean Squared Error: 2093550683154.5171
Mean Absolute Error: 1055461.2615
R-squared: 0.5858


In [ ]:
import numpy as np

random_idx = np.random.randint(0, len(X_test))

sample = X_test.iloc[random_idx:random_idx+1]  # giữ dạng DataFrame
true_label = y_test.iloc[random_idx]

def predict_value(model, X_sample, y_sample):
    predicted_label = model.predict(X_sample)[0]

    print(f"True label: {y_sample}")
    print(f"Predicted label: {predicted_label}")

In [60]:
print("Linear Regression")
predict_value(reg, sample, true_label)
print("ANN")
predict_value(ann_model, sample, true_label)
print("Random Forest")
predict_value(rf_model, sample, true_label)
print("XG Boost")
predict_value(xgb_model, sample, true_label)

Linear Regression
True label: 3500000
Predicted label: 2733512.804363784
ANN
True label: 3500000
Predicted label: 3582309.0
Random Forest
True label: 3500000
Predicted label: 3611144.305555555
XG Boost
True label: 3500000
Predicted label: 3881362.0


In [ ]:
# Chỉnh thêm để phân tích lý do các mô hình cho kết quả khác nhau

# Tạo một mẫu đầu vào
import numpy as np
import pandas as pd

# Sử dụng mẫu dữ liệu đã chuẩn hóa từ X_test
test_sample = X_test.iloc[0:1].copy()

# In thông tin về mẫu thử nghiệm
print("Dữ liệu mẫu:")
print(test_sample)

# So sánh kết quả giữa các mô hình với cùng một mẫu đầu vào
print("\nSo sánh dự đoán:")
print("=" * 50)

# Linear Regression
lr_pred = reg.predict(test_sample)[0]
print(f"Linear Regression: {lr_pred:.2f}")
print(f"Công thức: np.dot(X, weights) + bias")
print(f"Trọng số: {reg.weights}")
print(f"Bias: {reg.bias}")

# ANN
ann_pred = ann_model(torch.tensor(test_sample.values, dtype=torch.float32)).item()
print(f"\nANN: {ann_pred:.2f}")
print(f"Mô hình: 3 lớp ẩn (64-32-16 neurons)")

# Random Forest
rf_pred = rf_model.predict(test_sample)[0]
print(f"\nRandom Forest: {rf_pred:.2f}")
print(f"Tham số: n_estimators=100, max_depth=10")

# XGBoost
xgb_pred = xgb_model.predict(test_sample)[0]
print(f"\nXGBoost: {xgb_pred:.2f}")
print(f"Tham số: n_estimators=100, learning_rate=0.1, max_depth=6")

# Phân tích sự khác biệt
print("\n" + "=" * 50)
print("Phân tích sự khác biệt giữa các mô hình:")
print(f"Chênh lệch lớn nhất: {max(lr_pred, ann_pred, rf_pred, xgb_pred) - min(lr_pred, ann_pred, rf_pred, xgb_pred):.2f}")

In [ ]:
# Tạo và lưu các mô hình
import joblib
import os
import torch

# Tạo thư mục lưu trữ nếu chưa tồn tại
os.makedirs("models_ml", exist_ok=True)

# Lưu mô hình Linear Regression
joblib.dump(reg, "models_ml/linear_regression_model.joblib")

# Lưu mô hình ANN
torch.save(ann_model.state_dict(), "models_ml/ann_model.pth")

# Lưu mô hình Random Forest
joblib.dump(rf_model.model, "models_ml/random_forest_model.joblib")

# Lưu mô hình XGBoost
joblib.dump(xgb_model.model, "models_ml/xgboost_model.joblib")

# Lưu các thành phần tiền xử lý
# Thông tin cột
column_info = {
    'target_col': target_col,
    'continuous_cols': list(continuous_cols),
    'categorical_cols': list(categorical_cols),
    'binary_cols': ['pool', 'garden', 'security', 'renovated'],  # Điều chỉnh theo dữ liệu thực tế
    'input_features': list(X_train.columns)
}
joblib.dump(column_info, "models_ml/column_info.joblib")

# Lưu scaler
joblib.dump(scaler, "models_ml/scaler.joblib")

# Lưu label encoders
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    label_encoders[col].fit(housing[col])
joblib.dump(label_encoders, "models_ml/label_encoders.joblib")

print("Đã lưu tất cả các mô hình và thành phần tiền xử lý vào thư mục 'models_ml'")